In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline 
from sklearn import metrics
from sklearn.metrics import mean_squared_error as MSE
from sklearn import preprocessing
from sklearn.cluster import DBSCAN
from scipy.stats import mode
from sklearn.preprocessing import StandardScaler,PowerTransformer

from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.cluster import AgglomerativeClustering
import lightgbm as lgb

import xgboost as xgb
from sklearn.metrics import mean_squared_error

import sklearn
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)


# from sklearn import cross_validation, metrics   #Additional scklearn functions
# from sklearn.grid_search import GridSearchCV   #Perforing grid search


#Import Files 


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# # Import data
# data_songs=pd.read_csv('/content/drive/MyDrive/datacontest/songs.csv')
# data_labels=pd.read_csv('/content/drive/MyDrive/datacontest/song_labels.csv')
# data_train=pd.read_csv('/content/drive/MyDrive/datacontest/train.csv')
# data_saveforlater=pd.read_csv('/content/drive/MyDrive/datacontest/save_for_later.csv')
# df_test=pd.read_csv('/content/drive/MyDrive/datacontest/test.csv')

# df = pd.DataFrame(np.arange(0,len(df_test),1),columns=['test_row_id'])
# df['customer_id']=df_test['customer_id']
# df['song_id']=df_test['song_id']



In [ ]:
# Import data
data_songs=pd.read_csv('songs.csv')
data_labels=pd.read_csv('song_labels.csv')
data_train=pd.read_csv('train.csv')
data_saveforlater=pd.read_csv('save_for_later.csv')
df_test=pd.read_csv('test.csv')


df = pd.DataFrame(np.arange(0,len(df_test),1),columns=['test_row_id'])
df['customer_id']=df_test['customer_id']
df['song_id']=df_test['song_id']




In [ ]:
df_cust_map=pd.DataFrame(np.unique(np.array(pd.concat([df_test['customer_id'],data_train['customer_id']]))),columns=['customer_id'])

In [ ]:
df_song_map=pd.DataFrame(np.unique(np.array(pd.concat([df_test['song_id'],data_train['song_id']]))),columns=['song_id'])

# Labels Confidence Implementation


In [ ]:
df_train = data_train


In [ ]:
data_labels_fixed = data_labels.groupby("platform_id").max()
data_labels_fixed['total_count']=data_labels[['platform_id','count']].groupby("platform_id").sum()
data_labels_fixed['label_confidence']=data_labels_fixed['count']/data_labels_fixed['total_count']

In [ ]:
del data_labels_fixed['count']

In [ ]:
# Number of labels to consider
numberoflabels=50

data_labels_fixed = data_labels_fixed.reset_index()
good_labels=np.array(data_labels_fixed['label_id'].value_counts().reset_index().iloc[0:numberoflabels]['index'])

for i in range(10000):
    if data_labels_fixed['label_id'][i] not in good_labels:
        data_labels_fixed.at[i,'label_id']=-1

# DEAL WITH NANs

In [ ]:
data_songs['released_year'].fillna(2012,inplace=True)

In [ ]:

# data_songs['language']=data_songs['language'].astype(str)
# data_songs['language']=data_songs['language'].replace('en','eng')

# data_songs['language']=data_songs['language'].replace('en-US','eng')
# data_songs['language']=data_songs['language'].replace('en-GB','eng')
# data_songs['language']=data_songs['language'].replace('en-CA','eng')

data_songs['language'].fillna('nan',inplace=True)

In [ ]:
data_songs['number_of_comments'].fillna(4842,inplace=True)


# DJ COPY

In [ ]:
# df_train_unmerged = df_train.copy()
# df_test_unmerged = df_test.copy()

# Merging Data


In [ ]:
# UNCOMMENT WHEN LABELS READY


data_songs=pd.merge(
    data_songs,
    data_labels_fixed,
    how="left",
    on='platform_id',
)

In [ ]:
df_train=pd.merge(
    df_train,
    data_songs,
    how="left",
    on='song_id',
)
df_test=pd.merge(
    df_test,
    data_songs,
    how="left",
    on='song_id',
)
data_saveforlater=pd.merge(
    data_saveforlater,
    data_songs,
    how="left",
    on='song_id',
)

# Get Customer Perferences

In [ ]:
df_cust_map=pd.merge(
    df_cust_map,
    data_saveforlater[['customer_id','released_year']].groupby('customer_id').median().reset_index(),
    how="left",
    on='customer_id',
)


In [ ]:
df_cust_map=pd.merge(
    df_cust_map,
    data_saveforlater[['customer_id','number_of_comments']].groupby('customer_id').median().reset_index(),
    how="left",
    on='customer_id',
)


In [ ]:
def tempgrouper(x):
  try: 
    return x.value_counts().index[0] 
  except: 
    return np.nan

In [ ]:
def tempconfidence(x):
  try:
    return x.value_counts()[0]/sum(x.value_counts())
  except:
    return np.nan

In [ ]:
df_cust_map=pd.merge(
    df_cust_map,
    data_saveforlater[['customer_id','language']].groupby('customer_id').agg(lambda x:tempgrouper(x)).reset_index(),
    how="left",
    on='customer_id',
)

In [ ]:
df_temp=data_saveforlater[['customer_id','language']].groupby('customer_id').agg(lambda x:tempconfidence(x)).reset_index()
df_temp.rename(columns={"language": "language_confidence"},inplace=True)


df_cust_map=pd.merge(
    df_cust_map,
    df_temp,
    how="left",
    on='customer_id',
)
del df_temp

In [ ]:
df_temp=data_saveforlater[['customer_id','language']].groupby('customer_id').agg(lambda x:sum(x.value_counts())).reset_index()
df_temp.rename(columns={"language": "language_total"},inplace=True)


df_cust_map=pd.merge(
    df_cust_map,
    df_temp,
    how="left",
    on='customer_id',
)
del df_temp

In [ ]:
df_cust_map.rename(columns={"released_year": "released_year_c","number_of_comments":"number_of_comments_c","language":"language_c"},inplace=True)


In [ ]:
df_cust_map=pd.merge(
    df_cust_map,
    data_saveforlater[['customer_id','label_id']].groupby('customer_id').agg(lambda x:tempgrouper(x)).reset_index(),
    how="left",
    on='customer_id',
)

In [ ]:
df_cust_map.rename(columns={"released_year": "released_year_c","number_of_comments":"number_of_comments_c","language":"language_c"},inplace=True)


# Merge Customer data and form new features

In [ ]:

df_train=pd.merge(
    df_train,
    df_cust_map,
    how="left",
    on='customer_id',
)
df_test=pd.merge(
    df_test,
    df_cust_map,
    how="left",
    on='customer_id',
)

In [ ]:
df_train['language_TF']=df_train['language']==df_train['language_c']
df_test['language_TF']=df_test['language']==df_test['language_c']

In [ ]:
df_train['released_year_diff']=df_train['released_year']-df_train['released_year_c']
df_test['released_year_diff']=df_test['released_year']-df_test['released_year_c']

In [ ]:
df_train['number_of_comments_diff']=df_train['number_of_comments']-df_train['number_of_comments_c']
df_test['number_of_comments_diff']=df_test['number_of_comments']-df_test['number_of_comments_c']

In [ ]:
df_train['label_TF']=df_train['label_id_x']==df_train['label_id_y']
df_test['label_TF']=df_test['label_id_x']==df_test['label_id_y']

In [ ]:
del df_train['released_year']
del df_test['released_year']

del df_train['released_year_c']
del df_test['released_year_c']

del df_train['number_of_comments']
del df_test['number_of_comments']

del df_train['number_of_comments_c']
del df_test['number_of_comments_c']

del df_train['language']
del df_test['language']

del df_train['language_c']
del df_test['language_c']

del df_train['label_id_x']
del df_test['label_id_x']

del df_train['label_id_y']
del df_test['label_id_y']



# Getting Song data

In [ ]:
df_temp=data_saveforlater[['customer_id',	'song_id'	]].groupby('song_id').count().reset_index()
df_temp.rename(columns={"customer_id": "count_saved"},inplace=True)


df_song_map=pd.merge(
    df_song_map,
    df_temp,
    how="left",
    on='song_id',
)
del df_temp

In [ ]:

df_train=pd.merge(
    df_train,
    df_song_map,
    how="left",
    on='song_id',
)
df_test=pd.merge(
    df_test,
    df_song_map,
    how="left",
    on='song_id',
)

# Mean Encoding

In [ ]:
Mean_cust_score = df_train.groupby(['customer_id'])['score'].mean().to_dict()
df_train['customer_means'] =  df_train['customer_id'].map(Mean_cust_score)
df['score_mean']=df['customer_id'].astype(str).map(Mean_cust_score)

df_test['customer_means'] =  df_test['customer_id'].map(Mean_cust_score)

Mean_song_score = df_train.groupby(['song_id'])['score'].mean().to_dict()
df_train['song_means'] =  df_train['song_id'].map(Mean_song_score)
df_test['song_means'] =  df_test['song_id'].map(Mean_song_score)



# Label Encoding


In [ ]:
def labelencodethis2(df_train,df_test,column_name):
  le = preprocessing.LabelEncoder()
  fitter = le.fit(np.unique(np.array(pd.concat([df_test[column_name].astype(str),df_train[column_name].astype(str)]))))
  df_train[column_name] = fitter.transform(df_train[column_name].astype(str))
  df_test[column_name] = fitter.transform(df_test[column_name].astype(str))
  df_train[column_name]=df_train[column_name].astype(int)
  df_test[column_name]=df_test[column_name].astype(int)

def labelencodethis3(df_train,df_test,data_saveforlater,column_name):
  le = preprocessing.LabelEncoder()
  fitter = le.fit(np.unique(np.array(pd.concat([df_test[column_name].astype(str),df_train[column_name].astype(str),data_saveforlater[column_name].astype(str)]))))
  df_train[column_name] = fitter.transform(df_train[column_name].astype(str))
  df_test[column_name] = fitter.transform(df_test[column_name].astype(str))
  data_saveforlater[column_name] = fitter.transform(data_saveforlater[column_name].astype(str))

  df_train[column_name]=df_train[column_name].astype(int)
  df_test[column_name]=df_test[column_name].astype(int)
  data_saveforlater[column_name]=data_saveforlater[column_name].astype(int)



In [ ]:
labelencodethis3(df_train,df_test,data_saveforlater,'customer_id')
labelencodethis3(df_train,df_test,data_saveforlater,'song_id')
labelencodethis2(df_train,df_test,'language_TF')
labelencodethis2(df_train,df_test,'label_TF')


In [ ]:
del df_test['platform_id']
del df_train['platform_id']

# Random SHit

In [ ]:
df_train['released_year_diff']=np.abs(df_train['released_year_diff'])
df_test['released_year_diff']=np.abs(df_test['released_year_diff'])

df_train['number_of_comments_diff']=np.abs(df_train['number_of_comments_diff'])
df_test['number_of_comments_diff']=np.abs(df_test['number_of_comments_diff'])


In [ ]:
df_train['score_act']=df_train['score']-df_train['customer_means']


# Catboost

In [ ]:
df_trainac=df_train[['customer_id','song_id','score_act','language_confidence',	'language_total',	'released_year_diff',	'number_of_comments_diff',	'language_TF',	'count_saved','label_TF','label_confidence']]
df_testac=df_test[['customer_id','song_id','language_confidence',	'language_total',	'released_year_diff',	'number_of_comments_diff',	'language_TF',	'count_saved','label_TF','label_confidence']]


In [ ]:
df_trainac['language_confidence']=df_trainac['language_confidence'].fillna(0)

df_trainac['language_total']=df_trainac['language_total'].fillna(19)
df_trainac['released_year_diff']=df_trainac['released_year_diff'].fillna(47.67)
df_trainac['number_of_comments_diff']=df_trainac['number_of_comments_diff'].fillna(7385.0  )

df_testac['language_confidence']=df_testac['language_confidence'].fillna(0)
df_testac['language_total']=df_testac['language_total'].fillna(19)
df_testac['released_year_diff']=df_testac['released_year_diff'].fillna(47.67)
df_testac['number_of_comments_diff']=df_testac['number_of_comments_diff'].fillna( 7385.0)


In [ ]:
categorical_features_indices = [0,1]

df_actual_train = df_trainac[df_trainac.columns[df_trainac.columns != 'score_act']].copy()


X_train_catboost, X_validation_catboost, y_train_catboost, y_validation_catboost = train_test_split(df_actual_train, df_trainac['score_act'],test_size=0.2, random_state=123)



modelcat=CatBoostRegressor(iterations=500, depth=8, learning_rate=0.2, loss_function='RMSE', allow_writing_files=False,random_seed=123)

modelcat.fit(X_train_catboost, y_train_catboost,cat_features=categorical_features_indices,eval_set=(X_validation_catboost, y_validation_catboost),verbose=False)

In [ ]:
pred=modelcat.predict(df_testac)
df['catboost']=pred+df['score_mean']
df['catboost']=df['catboost'].fillna(df['score_mean'])


In [ ]:
del df_testac
del df_trainac
del modelcat

# XGBoost


In [ ]:
df_train_xgb=df_train[['score_act','customer_means', 'language_confidence',	'language_total',	'released_year_diff',	'number_of_comments_diff',	'count_saved', 'label_confidence','song_means']]
df_test_xgb=df_test[['customer_means', 'language_confidence',	'language_total',	'released_year_diff',	'number_of_comments_diff',	'count_saved', 'label_confidence','song_means']]

predictions_xgb = xgb.DMatrix(df_test_xgb)

In [ ]:
y_xgb = df_train_xgb['score_act']
x_xgb = df_train_xgb.drop('score_act',axis = 1)

In [ ]:
x_train_xgb, x_test_xgb, y_train_xgb, y_test_xgb = train_test_split(x_xgb, y_xgb, test_size=0.25)
dtrain = xgb.DMatrix(x_train_xgb, label=y_train_xgb)
dtest = xgb.DMatrix(df_test_xgb)


In [ ]:
optimumparams = {'colsample_bytree': 0.8969777504856158,
 'gamma': 0.07867280134809351,
 'max_depth': 6}

In [ ]:
modelxgb = xgb.train(optimumparams, dtrain, num_boost_round=250)


In [ ]:
import joblib

In [ ]:
# Predict on testing and training set
# training
#bst = xgb.train(param, dtrain, num_round)

#save model
joblib.dump(modelxgb, 'xgb_model.dat')
modelxgb.__del__()

#load saved model
modelxgb = joblib.load('xgb_model.dat')

pred_xgb = modelxgb.predict(dtest)

In [ ]:
import os
os.remove('xgb_model.dat')

In [ ]:
xgbscores = pd.DataFrame(pred_xgb)

In [ ]:
df['xgboostscore'] = xgbscores.copy()

,test_row_id,customer_id,song_id,score_mean,xgboostscore
0,0,J24644,5304,3.566667,-0.482059
1,1,G29701,123,3.722222,-0.084154
2,2,M29977,11,4.000000,0.104349
3,3,H50752,1445,3.600000,0.058483
4,4,J42394,271,4.476190,0.068271


In [ ]:
df['xgboost'] = df['xgboostscore']+df['score_mean']

In [ ]:
df['xgboost']=df['xgboost'].fillna(df['score_mean'])

In [ ]:
del df['xgboostscore']

In [ ]:
del df_train_xgb
del df_test_xgb

# LIGHTGBM


In [ ]:

df_train_lgbm=df_train[['score_act','customer_id','song_id', 'language_confidence',	'language_total',	'released_year_diff',	'number_of_comments_diff',	'language_TF',	'count_saved','label_TF','label_confidence','song_means']]
df_test_lgbm=df_test[['customer_id','song_id', 'language_confidence',	'language_total',	'released_year_diff',	'number_of_comments_diff',	'language_TF',	'count_saved','label_TF','label_confidence','song_means']]

In [ ]:
y_lgbm = df_train_lgbm['score_act']
x_lgbm = df_train_lgbm.drop('score_act',axis = 1)

In [ ]:
opt_params={'bagging_fraction': 0.9664821006536182,
 'boost_from_average': False,
 'feature_fraction': 0.5056948113060237,
 'is_unbalance': True,
 'learning_rate': 0.5114358817391321,
 'max_bin': 90,
 'max_depth': 12,
 'metric': 'rmse',
 'min_data_in_leaf': 29,
 'min_sum_hessian_in_leaf': 98.78899084527525,
 'num_leaves': 32,
 'objective': 'regression',
 'subsample': 0.6912001887307938}

In [ ]:
x_train_lgbm, x_test_lgbm, y_train_lgbm, y_test_lgbm = train_test_split(x_lgbm, y_lgbm, test_size=0.2, random_state=42)

In [ ]:
gbm = lgb.LGBMRegressor(**opt_params)

In [ ]:
gbm.fit(x_train_lgbm, y_train_lgbm,
        eval_set=[(x_test_lgbm, y_test_lgbm)],
        eval_metric='l1',
        early_stopping_rounds=1000,
        categorical_feature = [0,1,6,8],verbose=0)

In [ ]:
y_pred_lgbm = gbm.predict(df_test_lgbm, num_iteration=gbm.best_iteration_)

In [ ]:
df['lightgbm']=y_pred_lgbm+df['score_mean']
df['lightgbm']=df['lightgbm'].fillna(df['score_mean'])

In [ ]:
del df_train_lgbm
del df_test_lgbm

Temporary

In [ ]:
df['score']=(df['catboost']+df['lightgbm']+df['xgboost'])/3
df['score']=df['score'].clip(1,5)
df['score']=df['score'].round(0)

df[['test_row_id','score']].to_csv("predictions.csv",index=False)


# Submit


In [ ]:
print("Saved the csv file succesfully")